In [1]:
import torch
from pytorch_transformers import BertTokenizer

from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.loss import abs_loss
from models.model_builder import AbsSummarizer
from models.predictor import build_predictor
from models.trainer import build_trainer
from others.logging import logger, init_logger

In [3]:
# import os
# import glob
# model_path = "./checkpoints/abs"
# cp_file = sorted(glob.glob(os.path.join(model_path, 'model_step_*.pt')))
# cp = cp_file[0]

In [9]:
device = "cpu"
model_path = './checkpoints/abs/model_step_18000.pt'
# checkpoint = torch.load(cp, map_location=torch.device('cpu'))
checkpoint = torch.load(cp, map_location=lambda storage, loc: storage)

In [13]:
args = {}
model_flags = ['hidden_size', 'ff_size', 'heads', 'emb_size', 'enc_layers', 'enc_hidden_size', 'enc_ff_size',
               'dec_layers', 'dec_hidden_size', 'dec_ff_size', 'encoder', 'ff_actv', 'use_interval']

In [31]:
opt = vars(checkpoint['opt'])

In [54]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-task", default='ext', type=str, choices=['ext', 'abs'])
parser.add_argument("-encoder", default='bert', type=str, choices=['bert', 'baseline'])
parser.add_argument("-mode", default='train', type=str, choices=['train', 'validate', 'test'])
parser.add_argument("-bert_data_path", default='../bert_data_new/cnndm')
parser.add_argument("-model_path", default='../models/')
parser.add_argument("-result_path", default='../results/cnndm')
parser.add_argument("-temp_dir", default='../temp')

parser.add_argument("-batch_size", default=140, type=int)
parser.add_argument("-test_batch_size", default=200, type=int)

parser.add_argument("-max_pos", default=512, type=int)
parser.add_argument("-use_interval", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-large", type=bool, nargs='?',const=True,default=False)
parser.add_argument("-load_from_extractive", default='', type=str)

parser.add_argument("-sep_optim", type=bool, nargs='?',const=True,default=False)
parser.add_argument("-lr_bert", default=2e-3, type=float)
parser.add_argument("-lr_dec", default=2e-3, type=float)
parser.add_argument("-use_bert_emb", type=bool, nargs='?',const=True,default=False)

parser.add_argument("-share_emb", type=bool, nargs='?', const=True, default=False)
parser.add_argument("-finetune_bert", type=bool, nargs='?', const=True, default=True)
parser.add_argument("-dec_dropout", default=0.2, type=float)
parser.add_argument("-dec_layers", default=6, type=int)
parser.add_argument("-dec_hidden_size", default=768, type=int)
parser.add_argument("-dec_heads", default=8, type=int)
parser.add_argument("-dec_ff_size", default=2048, type=int)
parser.add_argument("-enc_hidden_size", default=512, type=int)
parser.add_argument("-enc_ff_size", default=512, type=int)
parser.add_argument("-enc_dropout", default=0.2, type=float)
parser.add_argument("-enc_layers", default=6, type=int)

parser.add_argument('-visible_gpus', default='-1', type=str)
parser.add_argument('-gpu_ranks', default='0', type=str)
parser.add_argument('-log_file', default='../logs/cnndm.log')
parser.add_argument('-seed', default=666, type=int)

parser.add_argument("-test_all", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-test_from", default='')
parser.add_argument("-test_start_from", default=-1, type=int)

parser.add_argument("-train_from", default='')
parser.add_argument("-report_rouge", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-block_trigram", type=bool, nargs='?', const=True, default=True)

args = parser.parse_args()
args.gpu_ranks = [int(i) for i in range(len(args.visible_gpus.split(',')))]
args.world_size = len(args.gpu_ranks)

In [ ]:
# args.task = abs 
# args.mode = "validate" 
args.batch_size = 3000 
args.test_batch_size = 500 
args.bert_data_path = "../bert_data/bert_data_cnndm_final/cnndm"  
# args.log_file = "../logs/val_abs_bert_cnndm"
# args.model_path = "../models/absext/model_step_148000.pt" 
args.sep_optim = True 
args.use_interval = True 
args.visible_gpus = -1 
args.max_pos = 512 
args.max_length = 200 
args.alpha = 0.95 
args.min_length = 50 
args.result_path = "../logs/abs_bert_cnndm" 

In [58]:
for k in opt.keys():
    if (k in model_flags):
        setattr(args, k, opt[k])
print(args)

Namespace(batch_size=140, bert_data_path='../bert_data_new/cnndm', block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', finetune_bert=True, gpu_ranks=[0], large=False, load_from_extractive='', log_file='../logs/cnndm.log', lr_bert=0.002, lr_dec=0.002, max_pos=512, mode='train', model_path='../models/', report_rouge=True, result_path='../results/cnndm', seed=666, sep_optim=False, share_emb=False, task='ext', temp_dir='../temp', test_all=True, test_batch_size=200, test_from='', test_start_from=-1, train_from='', use_bert_emb=False, use_interval=True, visible_gpus='-1', world_size=1)


In [60]:
model = AbsSummarizer(args, device, checkpoint)
model.eval()

RuntimeError: Error(s) in loading state_dict for AbsSummarizer:
	Missing key(s) in state_dict: "decoder.embeddings.weight", "decoder.pos_emb.pe", "decoder.transformer_layers.0.mask", "decoder.transformer_layers.0.self_attn.linear_keys.weight", "decoder.transformer_layers.0.self_attn.linear_keys.bias", "decoder.transformer_layers.0.self_attn.linear_values.weight", "decoder.transformer_layers.0.self_attn.linear_values.bias", "decoder.transformer_layers.0.self_attn.linear_query.weight", "decoder.transformer_layers.0.self_attn.linear_query.bias", "decoder.transformer_layers.0.self_attn.final_linear.weight", "decoder.transformer_layers.0.self_attn.final_linear.bias", "decoder.transformer_layers.0.context_attn.linear_keys.weight", "decoder.transformer_layers.0.context_attn.linear_keys.bias", "decoder.transformer_layers.0.context_attn.linear_values.weight", "decoder.transformer_layers.0.context_attn.linear_values.bias", "decoder.transformer_layers.0.context_attn.linear_query.weight", "decoder